# SQL Workshop

## Introductions

* Name
* Background/Role
* Prior Experiences
* Expectations
* Hobby

* Michael Burgess
    * michael.burgess@decoded.com
* Head of Technical Solutions -- design commerical educational programmes
    * Data, Analytics, & AI -- Physics, Contracting (defence, telefony, mobile)
* Arguing, Podcasts, Youtube


* John
    * Fidelity, Infrastructure Architect
        * Java, SQL (prior experience)
    * Applying SQL within jupyter env. 
    * Travel, boy scouting, camping
* Sofiah
    * Fidelity, Brunei, BA in Innovation/AI
        * no prior experience
        * last day of work 
    * Use SQL
    * Theatre, Music, Orchestra
* Philip
    * Service Reporting, Service Management Reporting
    * Some prior SQL, connecting & pulling -- no analytics
        * python & SQL
    * Rugby, Sci-Fi
    * Be Bold
* Gema (BGC)
    * Internal Audit (Snr Audit Analyst)
        * minimial prior exp. with coding, sql
        * apply within testing analyst
    * Galleries, Creative... 

By default, the `sqlite3` library is included in python,

In [1]:
import sqlite3

The `sqlalchemy` library allows you to work with *any* database,

In [2]:
from sqlalchemy import create_engine

In [12]:
db = create_engine("sqlite:///newdb2.db")
con = db.connect()

---

## Using Pandas with SQL

Pandas is like Excel... its just a spreadsheet interface. Pandas always needs to "open" ("read") a dataset.

Sources: csv, json... or database connection.

In [13]:
import pandas as pd

In [14]:
data = pd.DataFrame({
    "Age": [10, 20, 30],
    "Name": ["Alice", "Eve", "Bob"]
})

In [15]:
data

,Age,Name
0,10,Alice
1,20,Eve
2,30,Bob


In [16]:
data.to_sql('people',con, index=False)

3

In [17]:
pd.read_sql("SELECT * FROM people", con)

,Age,Name
0,10,Alice
1,20,Eve
2,30,Bob


---

### Template

In [18]:
query = """
    SELECT *
    FROM people
"""

pd.read_sql( query , con)

,Age,Name
0,10,Alice
1,20,Eve
2,30,Bob


---

In [22]:
query = """
    SELECT Age
    FROM people
"""

pd.read_sql( query , con)

,Age
0,10
1,20
2,30


In [23]:
query = """
    SELECT AVG(Age)
    FROM people
"""

pd.read_sql( query , con)

,AVG(Age)
0,20.0


In [20]:
query = """
    SELECT Age, Name
    FROM people
"""

pd.read_sql( query , con)

,Age,Name
0,10,Alice
1,20,Eve
2,30,Bob


In [21]:
query = """
    SELECT Age, Name
    FROM people
    ORDER BY Name
"""

pd.read_sql( query , con)

,Age,Name
0,10,Alice
1,30,Bob
2,20,Eve


In [26]:
query = """
    SELECT 
        AVG(Age) AS AvgAge
    FROM people
"""

pd.read_sql( query , con)

,AvgAge
0,20.0
